In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Big Query

In [2]:
from google.cloud import bigquery

In [3]:
client=bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In [4]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [6]:
tables = list(client.list_tables(dataset))

In [7]:
for table in tables:
    print(table.table_id)

comments
full
full_201510
stories


In [8]:
table_ref = dataset_ref.table("full")
table = client.get_table(table_ref)

In [10]:
table.description

'A full daily update of all the stories and comments in Hacker News.'

In [11]:
table.num_rows

27708712

## Table Schema

In [12]:
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', 'Story title', (), None),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', (), None),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', (), None),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', (), None),
 SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", (), None),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', (), None),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', (), None),
 SchemaField('type', 'STRING', 'NULLABLE', 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', (), None),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", (), None),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', (), None),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', (), None),
 SchemaField

In [13]:
client.list_rows(table,max_results=5)

In [14]:
client.list_rows(table,max_results=5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.


,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,&gt;I&#x27;m done with this conversation since...,None,noobly,NaN,1541023247,2018-10-31 22:00:47+00:00,comment,18349611,18348253,NaN,NaN,None
1,None,None,What do you mean by sync being hard to impleme...,None,e_proxus,NaN,1434378064,2015-06-15 14:21:04+00:00,comment,9719409,9719293,NaN,NaN,None
2,None,None,"This is pretty dumb and naive, and loaded with...",None,Retra,NaN,1434378063,2015-06-15 14:21:03+00:00,comment,9719408,9719205,NaN,NaN,None
3,None,None,5. Biker not using lights at night &quot;becau...,None,rjsw,NaN,1434378022,2015-06-15 14:20:22+00:00,comment,9719401,9719372,NaN,NaN,None
4,None,None,But they are all either very inexpensive or th...,None,g8gggu89,NaN,1434378020,2015-06-15 14:20:20+00:00,comment,9719400,9719006,NaN,NaN,None


In [15]:
client.list_rows(table, selected_fields=table.schema[:1], max_results = 5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.


,title
0,None
1,None
2,None
3,None
4,None


## Select, From and Where in Bigquery

In [4]:
dataset_ref=(client.dataset("openaq", project = "bigquery-public-data"))
dataset=client.get_dataset(dataset_ref)

tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

global_air_quality


In [5]:
table_ref = dataset_ref.table("global_air_quality")
table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  after removing the cwd from sys.path.


,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


In [20]:
query="""SELECT city
         FROM `bigquery-public-data.openaq.global_air_quality`
         WHERE country = 'US'"""

In [22]:
client=bigquery.Client()
query_job = client.query(query)

us_cities = query_job.to_dataframe()

us_cities

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,city
0,BROWN
1,BROWN
2,BROWN
3,BROWN
4,Houston
...,...
3718,Atlanta
3719,Mammoth Cave NP
3720,Las Cruces
3721,MARTIN


In [1]:
us_cities.city.value_counts().head()

NameError: name 'us_cities' is not defined

Since there is a limit of 5TB, you can first calculate the size of the query:

In [6]:
query="""SELECT city,country
         FROM `bigquery-public-data.openaq.global_air_quality`
         WHERE country = 'US'"""

dry_run_config = bigquery.QueryJobConfig(dry_run = True)

dry_run_query_job=client.query(query,job_config = dry_run_config)

print(f'This query will process {dry_run_query_job.total_bytes_processed} bytes')

This query will process 362151 bytes


In [8]:
print(f'This query will process {(dry_run_query_job.total_bytes_processed)/1000000} mb')

This query will process 0.362151 mb


In [10]:
print(f'This query will process {(dry_run_query_job.total_bytes_processed)/1000000000} gb')

This query will process 0.000362151 gb


You can also specify a parameter when running the query to limit how much data you are willing to scan. Here's an example with a low limit.

In [15]:
# Only run the query if it's less than 1 MB
ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)

# Set up the query (will only run if it's less than 1 MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
safe_query_job.to_dataframe()

InternalServerError: 500 Query exceeded limit for bytes billed: 1000000. 10485760 or higher required.

(job ID: cebbf7f8-c5f4-48d5-a181-989344505592)

                  -----Query Job SQL Follows-----                  

    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:SELECT city,country
   2:         FROM `bigquery-public-data.openaq.global_air_quality`
   3:         WHERE country = 'US'
    |    .    |    .    |    .    |    .    |    .    |    .    |

In [16]:
# Only run the query if it's less than 1 MB
ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=100000000)

# Set up the query (will only run if it's less than 1 MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
safe_query_job.to_dataframe()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,city,country
0,BROWN,US
1,BROWN,US
2,BROWN,US
3,BROWN,US
4,Houston,US
...,...,...
3718,Atlanta,US
3719,Mammoth Cave NP,US
3720,Las Cruces,US
3721,MARTIN,US


The query gets cancelled if the limit of 1 MB exceeds. However, we can increase the limit to run the query successfully!

In [20]:
# Only run the query if it's less than 1 GB
ONE_GB = 1000*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

# Set up the query (will only run if it's less than 1 GB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
job_post_scores = safe_query_job.to_dataframe()

# Print average score for job posts
job_post_scores.score.mean()

AttributeError: 'DataFrame' object has no attribute 'score'